In [174]:
# Import necessary libraries
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo
from scipy.ndimage import gaussian_filter1d
from scipy.stats import mode

In [175]:
class CTG:
    
    # Example: Assume 'fhr' is a NumPy array of heart rate values, 1 per second
    def compute_LB(fhr, window_size=600, smooth_sigma=5):
        """
        Computes the baseline FHR (LB) as the mode of the smoothed FHR over a 10-minute window.
        
        Parameters:
        - fhr: NumPy array of fetal heart rate (one value per second).
        - window_size: Size of the window (default 600 sec = 10 minutes).
        - smooth_sigma: Standard deviation for Gaussian smoothing.
    
        Returns:
        - LB (estimated baseline FHR)
        """
        # Step 1: Apply Gaussian smoothing
        smoothed_fhr = gaussian_filter1d(fhr, sigma=smooth_sigma)
    
        # Step 2: Compute mode over a sliding window
        baseline_values = []
        for i in range(len(fhr) - window_size + 1):
            window = smoothed_fhr[i:i + window_size]
            window_mode = mode(window, keepdims=True)[0][0]  # Most frequent value
            baseline_values.append(window_mode)
    
        # Step 3: Determine the most frequent baseline across all windows
        LB = mode(baseline_values, keepdims=True)[0][0]
    
        return round(LB)


    def compute_AC(fhr, LB):
        """
        Computes the number of accelerations per second (AC) from fetal heart rate (FHR) data.
    
        Parameters:
        - fhr: NumPy array of fetal heart rate values (one per second).
        - LB: Baseline fetal heart rate (computed separately).
    
        Returns:
        - AC: Number of accelerations per second.
        """
        acceleration_events = 0
        duration = len(fhr)  # Total duration in seconds
    
        i = 0
        while i < duration:
            # Check if FHR exceeds (LB + 15) to start an acceleration
            if fhr[i] >= LB + 15:
                start_time = i  # Mark the start of acceleration
                
                # Continue until FHR drops below (LB + 15)
                while i < duration and fhr[i] >= LB + 15:
                    i += 1
                
                # If the acceleration lasted at least 15 seconds, count it as an event
                if (i - start_time) >= 15:
                    acceleration_events += 1
            
            i += 1  # Move to the next time step
    
        # Compute AC as accelerations per second
        AC = acceleration_events / duration
        return round(AC, 3)



    def compute_DP(fhr, LB):
        """
        Computes the number of prolonged decelerations per second per second (DP) from fetal heart rate (FHR) data.
    
        Parameters:
        - fhr: NumPy array of fetal heart rate values (one per second).
        - LB: Baseline fetal heart rate (computed separately).
    
        Returns:
        - DP: Number of prolonged decelerations per second per second.
        """
        deceleration_events = 0
        duration = len(fhr)  # Total duration in seconds
    
        i = 0
        while i < duration:
            # Check if FHR exceeds (LB + 15) to start an acceleration
            if fhr[i] <= LB + 15:
                start_time = i  # Mark the start of acceleration
                
                # Continue until FHR drops below (LB + 15)
                while i < duration and fhr[i] <= LB + 15:
                    i += 1
                
                # If the deceleration lasted at least 2 minutes and less than 10 minutes, count it as an event
                if ((i - start_time) >= 120) & ((i - start_time) <= 600):
                    deceleration_events += 1
            
            i += 1  # Move to the next time step
    
        # Compute DP as prolonged decelerations per second
        DP = deceleration_events / duration
        return round(DP, 1)
        
            
    
    
    def compute_STV(fhr, segment_length=4, window_size=60):
        """
        Computes STV for each 1-minute window using the Arduini method.
    
        Parameters:
        - fhr: NumPy array of fetal heart rate values (one per second).
        - segment_length: Duration of each segment in seconds (default: 4 seconds).
        - window_size: Length of the window to compute STV (default: 60 seconds).
    
        Returns:
        - stv_values: NumPy array of STV values, one per minute.
        """
        total_minutes = len(fhr) // window_size  # Number of full 1-minute windows
        stv_values = []
    
        for i in range(total_minutes):
            start_idx = i * window_size
            end_idx = start_idx + window_size
            fhr_segment = fhr[start_idx:end_idx]  # Get 1-minute window
    
            # Split into segments of `segment_length` seconds
            segment_count = window_size // segment_length  
            fhr_segment = fhr_segment[:segment_count * segment_length]  # Trim excess
            reshaped_segment = fhr_segment.reshape(segment_count, -1)  # Reshape into segments
    
            # Compute average FHR for each segment (fixed variable)
            avg_fhr_per_segment = np.mean(reshaped_segment, axis=1)  # ✅ Corrected
    
            # Compute absolute differences between successive segment means
            segment_differences = np.abs(np.diff(avg_fhr_per_segment))
    
            # Compute STV for this minute
            STV = np.mean(segment_differences)
            stv_values.append(STV)
    
        return np.array(stv_values)  # Return as NumPy array



    def compute_ASTV(stv_values, threshold=3):
        """
        Computes ASTV (Abnormal Short-Term Variability) from STV values.
        
        Parameters:
        - stv_values: List or array of STV values (one per minute).
        - threshold: The STV threshold to define abnormal variability (default: 3 ms).
        
        Returns:
        - ASTV: Percentage of time (in minutes) with STV > threshold.
        """
        # Count how many minutes have STV > threshold
        abnormal_minutes = np.sum(stv_values < threshold)
        
        # Compute ASTV as percentage
        ASTV = round((abnormal_minutes / len(stv_values)) * 100)
        return ASTV



    def compute_ALTV(fhr, window_size=60, high_threshold=32, low_threshold=2, min_episode_duration=5):
        """
        Computes Long-Term Variability (LTV) from FHR and identifies high and low episodes.
        High and low episodes are counted only if they last for at least `min_episode_duration` minutes.
    
        Parameters:
        - fhr: NumPy array of fetal heart rate values (one per second).
        - window_size: Length of the window to compute LTV (default: 60 seconds).
        - high_threshold: Threshold in milliseconds for a high episode (default: 32 ms).
        - low_threshold: Threshold in milliseconds for a low episode (default: 2 ms).
        - min_episode_duration: Minimum duration of the episode in minutes (default: 5 minutes).
        
        Returns:
        - total_episodes: The total number of high and low episodes combined, where episodes last at least `min_episode_duration` minutes.
        """
        total_minutes = len(fhr) // window_size  # Number of full 1-minute windows
        high_episodes = 0
        low_episodes = 0
    
        # Track consecutive episodes
        consecutive_high = 0
        consecutive_low = 0
    
        for i in range(total_minutes):
            start_idx = i * window_size
            end_idx = start_idx + window_size
            fhr_segment = fhr[start_idx:end_idx]  # Get 1-minute window
    
            # Compute the absolute differences between successive FHR values (beat-to-beat variation)
            fhr_differences = np.abs(np.diff(fhr_segment))
    
            # Compute LTV for this minute (average of the differences)
            LTV = np.mean(fhr_differences)
    
            # Count high and low episodes with the duration check
            if LTV > high_threshold:
                consecutive_high += 1
                # If it's been at least 5 minutes, count it as a high episode
                if consecutive_high >= min_episode_duration:
                    high_episodes += 1
            else:
                consecutive_high = 0  # Reset consecutive high counter
    
            if LTV < low_threshold:
                consecutive_low += 1
                # If it's been at least 5 minutes, count it as a low episode
                if consecutive_low >= min_episode_duration:
                    low_episodes += 1
            else:
                consecutive_low = 0  # Reset consecutive low counter
    
        # Total episodes = sum of high and low episodes
        total_episodes = high_episodes + low_episodes
    
        return round(len(fhr) * total_episodes/1800)





    def features(fhr):
        """
        Computes several features from a fetal heart rate (FHR) signal.
    
        This function extracts a set of predefined features from the input FHR signal, 
        which is a time series of fetal heart rate values. The features include low-band 
        decomposition, autocorrelation, dynamic perturbation, mean, variance, short-term 
        variability, amplitude short-term variability, and long-term variability.
    
        The features are typically used for further analysis in the context of fetal health monitoring.
    
        Parameters:
        fhr (array-like): A numerical array or list of fetal heart rate values (in beats per minute, bpm).
                          Each value represents the fetal heart rate at a specific second in time.
                          The array must have at least 600 measurements (i.e., 10 minutes of data if sampled at 1Hz).
    
        Returns:
        dict: A dictionary containing the following features:
            - 'LB' : Baseline value (SisPorto), Numerical.
            - 'AC' : Accelerations (SisPorto), Numerical.
            - 'DP' : Prolonged decelerations, Numerical.
            - 'Mean' : Histogram mean, Numerical.
            - 'Variance' : Histogram variance, Numerical.
            - 'MSTV' : Mean value of short-term variability (SisPorto), Numerical.
            - 'ASTV' : Percentage of time with abnormal short-term variability, Numerical
            - 'ALTV' : Percentage of time with abnormal long-term variability, Numerical.
        """
        
        # Check if the FHR array is too short
        if len(fhr) < 600:
            return {
                'LB': None,
                'AC': None,
                'DP': None,
                'Mean': None,
                'Variance': None,
                'MSTV': None,
                'ASTV': None,
                'ALTV': None
            }
        else:
            # Compute features
            LB = compute_LB(fhr, window_size=600, smooth_sigma=5)
            AC = compute_AC(fhr, LB)
            DP = compute_DP(fhr, LB)
            Mean = round(np.mean(fhr))
            Variance = round(np.var(fhr))
            MSTV = round(np.mean(compute_STV(fhr, segment_length=4, window_size=60)), 1)
            ASTV = compute_ASTV(compute_STV(fhr, segment_length=4, window_size=60), threshold=3)
            ALTV = compute_ALTV(fhr, window_size=60, high_threshold=32, low_threshold=2, min_episode_duration=5)
    
            # Return features in a dictionary
            return {
                'LB': LB,
                'AC': AC,
                'DP': DP,
                'Mean': Mean,
                'Variance': Variance,
                'MSTV': MSTV,
                'ASTV': ASTV,
                'ALTV': ALTV
            }
    
        
